# Kaczmarek Kacper
Jakiś wstęp potem



---
# Procesowanie sygnału
#### 1. Zaciągnięcie danych z bazy https://physionet.org/content/gaitpdb/1.0.0/
#### 2. Zaczytanie danych
#### 3. Przedstawienie danych
---

### Zaczytanie danych

#### Zaczytanie metadanych

In [ ]:
import pandas as pd
patients = pd.read_csv('Data/demographics.txt', sep='\\t')

#### Zaczytanie danych o pacjentach

In [ ]:
import utils

records = utils.readData()

### Prezentacja danych

#### Przykład zdrowego i chorego i rozkład sił na lewej i prawej stopie

In [ ]:
import matplotlib.pyplot as plt
import utils

healthy_patient_id = "GaCo16"
parkinson_patient_id = "GaPt26"
sample_number = 240

healthy_record = records[healthy_patient_id]["data1"]
parkinson_record = records[parkinson_patient_id]["data1"]

utils.createComparisonPlot(healthy_record, parkinson_record, sample_number)



#### Każdy czujnik dla zdrowego i chorego

In [ ]:
import utils
healthy_patient_id = "GaCo16"

healthy_record = records[healthy_patient_id]["data1"]

utils.createAllSensorPlot(healthy_record, 'healthy_all_sensors', sample_number)

In [ ]:
import utils
parkinson_patient_id = "GaPt26"

parkinson_record = records[parkinson_patient_id]["data1"]

utils.createAllSensorPlot(parkinson_record, 'parkinson_all_sensors', sample_number)

#### Wyliczenie najmniejszej porcji danych chodu

In [ ]:
sample_len = float("inf")
for key, value in records.items():
    record = value['data1']
    sample_len = min(len(record.Time), sample_len)

print(sample_len)

### Pokazanie zachowań kroków na podstawie czujnika prawej stopy

In [ ]:
steps = {}
minVal = 30
for key, value in records.items():
    record = value['data1']
    idx = record.index[record.Force_Right < minVal].tolist()[0] + 1
    start = record[idx:].index[record.Force_Right[idx:] > minVal].tolist()[0]
    end = record[start:].index[record[start:].Force_Right < minVal].tolist()[0]
    step = record.Force_Right[start:end+1].tolist()
    steps[key] = [x / max(step) for x in step]

parkinsons = []
healthys = []
for key in steps:
    hoehnYahr = patients[patients['ID'] == key]['HoehnYahr'].values[0]
    parkinsons.append(key) if hoehnYahr > 0 else healthys.append(key)
    

plt.figure()
for key in parkinsons:
    plt.plot(steps[key])
plt.show()

plt.figure()
for key in healthys:
    plt.plot(steps[key])
plt.show()

In [ ]:
import numpy as np
steps = {}
minVal = 50
maxStep = float("-inf")
for key, value in records.items():
    record = value['data1']
    idx = record.index[record.Force_Right < minVal].tolist()[0] + 1
    start = record[idx:].index[record.Force_Right[idx:] > minVal].tolist()[0]
    end = record[start:].index[record[start:].Force_Right < minVal].tolist()[0]
    step = record.Force_Right[start:end+1].tolist()
    maxStep = max(maxStep, len(step))
    steps[key] = [x / max(step) for x in step]
print(maxStep)
def rescale(arr, maxStep):
    n = len(arr)
    return np.interp(np.linspace(0, n, maxStep), np.arange(n), arr)
parkinsons = []
healthys = []
for key in steps:
    hoehnYahr = patients[patients['ID'] == key]['HoehnYahr'].values[0]
    parkinsons.append(key) if hoehnYahr > 0 else healthys.append(key)
    

plt.figure()
for key in parkinsons:
    plt.plot(rescale(steps[key], maxStep))
plt.show()

plt.figure()
for key in healthys:
    plt.plot(rescale(steps[key], maxStep))
plt.show()

### Porównanie podczas liczenia do 7 w dół

In [ ]:
import numpy as np
steps = {}
minVal = 50
maxStep = float("-inf")
for key, value in records.items():
    if key.startswith('Ga') and 'data10' in value.keys():
        record = value['data10']
        idx = record.index[record.Force_Right < minVal].tolist()[0] + 1
        start = record[idx:].index[record.Force_Right[idx:] > minVal].tolist()[0]
        end = record[start:].index[record[start:].Force_Right < minVal].tolist()[0]
        step = record.Force_Right[start:end+1].tolist()
        maxStep = max(maxStep, len(step))
        steps[key] = [x / max(step) for x in step]
print(maxStep)
def rescale(arr, maxStep):
    n = len(arr)
    return np.interp(np.linspace(0, n, maxStep), np.arange(n), arr)
parkinsons = []
healthys = []
for key in steps:
    hoehnYahr = patients[patients['ID'] == key]['HoehnYahr'].values[0]
    parkinsons.append(key) if hoehnYahr > 0 else healthys.append(key)
    

plt.figure()
for key in parkinsons:
    plt.plot(rescale(steps[key], maxStep))
plt.show()

plt.figure()
for key in healthys:
    plt.plot(rescale(steps[key], maxStep))
plt.show()

#### Prezentacja typów falek

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt


points = 100
a = 4
w = 0

plt.figure()
vec_ricker = signal.ricker(points, a)
plt.plot(vec_ricker)

plt.savefig("plots/wavelet-type-ricker.pdf", format="pdf", bbox_inches="tight")

plt.figure()
vec_morlet2 = signal.morlet2(points, a, w)
plt.plot(vec_morlet2)

plt.savefig("plots/wavelet-type-morlet.pdf", format="pdf", bbox_inches="tight")


#### Wykresy transfomaty falkowej

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np

healthy_patient_id = "GaCo16"
parkinson_patient_id = "GaPt26"

healthy_record = records[healthy_patient_id]["data1"]
parkinson_record = records[parkinson_patient_id]["data1"]

sample_number = 1000

y_healthy = healthy_record.Force_Left.head(sample_number) - healthy_record.Force_Right.head(sample_number)
y_healthy = y_healthy.abs()
x_healthy = healthy_record.Time.head(sample_number)

y_parkinson = parkinson_record.Force_Left.head(sample_number) - parkinson_record.Force_Right.head(sample_number)
y_parkinson = y_parkinson.abs()
x_parkinson = parkinson_record.Time.head(sample_number)

width = 50
widths = np.arange(1, width)
cwtFunc = lambda: signal.cwt(y_healthy, signal.ricker, widths, dtype='float64')
path = "plots/healthy-wavelet"
utils.createWaveletPlot(x_healthy, cwtFunc, width, path, 'pdf')

widths = np.arange(1, width)
cwtFunc = lambda: signal.cwt(y_parkinson, signal.ricker, widths, dtype='float64')
path = "plots/parkinson-wavelet"
utils.createWaveletPlot(x_parkinson, cwtFunc, width, path, 'pdf')

Porównanie tranformaty falkowej morleta i rickera

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np

healthy_patient_id = "GaCo16"
parkinson_patient_id = "GaPt26"

healthy_record = records[healthy_patient_id]["data1"]

# sample_number = 4034
sample_number = 1000

y_healthy = healthy_record.Force_Left.head(sample_number)
x_healthy = healthy_record.Time.head(sample_number)

width = 50

w = 0
widths = np.arange(1, width)
cwtFunc = lambda: signal.cwt(y_healthy, signal.morlet2, widths, dtype='float64', w=w)
path = "plots/healthy-morlet"
utils.createWaveletPlot(x_healthy, cwtFunc, width, path, 'pdf')

widths = np.arange(1, width)
cwtFunc = lambda: signal.cwt(y_healthy, signal.ricker, widths, dtype='float64')
path = "plots/healthy-ricker"
utils.createWaveletPlot(x_healthy, cwtFunc, width, path, 'pdf')

### Klasyfikacja

In [ ]:
width = 50
widths = np.arange(1, width)
howManyInSample = 500
w = 0.0
num_epochs = 500

### Scenariusz 1 - Pojedyncze czujniki

In [ ]:
scenario1 = False
if scenario1:
    main_path = f'Datasets/Binary/AllSensors'
    utils.removeDatasetFolders(main_path)
    for wavelet in ['Ricker', 'Morlet']:
        utils.createDatasetFolders(f'{main_path}/{wavelet}')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

sensors = [
    (lambda x: x.L1, "L1"),
    (lambda x: x.L2, "L2"),
    (lambda x: x.L3, "L3"),
    (lambda x: x.L4, "L4"),
    (lambda x: x.L5, "L5"),
    (lambda x: x.L6, "L6"),
    (lambda x: x.L7, "L7"),
    (lambda x: x.L8, "L8"),
    # (lambda x: x.Force_Left, "Force_Left"),
    (lambda x: x.R1, "R1"),
    (lambda x: x.R2, "R2"),
    (lambda x: x.R3, "R3"),
    (lambda x: x.R4, "R4"),
    (lambda x: x.R5, "R5"),
    (lambda x: x.R6, "R6"),
    (lambda x: x.R7, "R7"),
    (lambda x: x.R8, "R8"),
    # (lambda x: x.Force_Right, "Force_Right")
    ]

i = 1
if scenario1:
    for id, value in records.items():
        print(f'{i}/{len(records)}: Processing ID = {id}')
        i = i + 1
        parkinson = utils.getParkinsonStatus(patients, id)
        hoehnYahr = utils.getHoehnYahrStatus(patients, id)
        record = value['data1']

        splits_list = [i for i in range(0, len(record), howManyInSample)]
        
        utils.createDatabase(record, id, sensors, splits_list, howManyInSample, main_path, parkinson, widths, w)
    print('-----------------END-----------------')

In [ ]:
if scenario1:
    for wavelet in ['Ricker', 'Morlet']:
        utils.createDatasets(f'{main_path}/{wavelet}')
    print('Datasets created')

#### Uczenie sieci

In [ ]:
import LearningUtils
import importlib
import ModelTrainer
from torchvision import models
import logging
import sys
importlib.reload(ModelTrainer)
importlib.reload(LearningUtils)


if scenario1:
    for wavelet in ['Ricker', 'Morlet']:
        print(f'Resnet for {wavelet}')
        imgdir = f"{main_path}/{wavelet}"
        # logger
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)
        fhandler = logging.FileHandler(f'{imgdir}/log.txt', mode='w')
        consoleHandler = logging.StreamHandler(sys.stdout)
        logger.addHandler(fhandler)
        logger.addHandler(consoleHandler)

        classes = LearningUtils.get_classes(imgdir)
        train_data_loader, validation_data_loader, test_data_loader = LearningUtils.prepare_data(imgdir, 128)
        model_trainer = ModelTrainer.ModelTrainer(classes, imgdir)
        resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model = model_trainer.train_resnet(resnet, train_data_loader, validation_data_loader, num_epochs=num_epochs)
        model.eval()
        LearningUtils.predict_image(model, test_data_loader, classes)

        logger.removeHandler(fhandler)
        logger.removeHandler(consoleHandler)

### Scenariusz 2 - Tylko sumaryczne

In [ ]:
scenario2 = False
if scenario2:
    main_path = f'Datasets/Binary/OnlySums'
    utils.removeDatasetFolders(main_path)
    for wavelet in ['Ricker', 'Morlet']:
        utils.createDatasetFolders(f'{main_path}/{wavelet}')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

sensors = [
    # (lambda x: x.L1, "L1"),
    # (lambda x: x.L2, "L2"),
    # (lambda x: x.L3, "L3"),
    # (lambda x: x.L4, "L4"),
    # (lambda x: x.L5, "L5"),
    # (lambda x: x.L6, "L6"),
    # (lambda x: x.L7, "L7"),
    # (lambda x: x.L8, "L8"),
    (lambda x: x.Force_Left, "Force_Left"),
    # (lambda x: x.R1, "R1"),
    # (lambda x: x.R2, "R2"),
    # (lambda x: x.R3, "R3"),
    # (lambda x: x.R4, "R4"),
    # (lambda x: x.R5, "R5"),
    # (lambda x: x.R6, "R6"),
    # (lambda x: x.R7, "R7"),
    # (lambda x: x.R8, "R8"),
    (lambda x: x.Force_Right, "Force_Right")
    ]

i = 1
if scenario2:
    for id, value in records.items():
        print(f'{i}/{len(records)}: Processing ID = {id}')
        i = i + 1
        parkinson = utils.getParkinsonStatus(patients, id)
        hoehnYahr = utils.getHoehnYahrStatus(patients, id)
        record = value['data1']

        splits_list = [i for i in range(0, len(record), howManyInSample)]

        utils.createDatabase(record, id, sensors, splits_list, howManyInSample, main_path, parkinson, widths, w)
    print('-----------------END-----------------')

In [ ]:
if scenario2:
    for wavelet in ['Ricker', 'Morlet']:
        utils.createDatasets(f'{main_path}/{wavelet}')
    print('Datasets created')

#### Uczenie sieci

In [ ]:
import LearningUtils
import importlib
import ModelTrainer
from torchvision import models
import logging
import sys
importlib.reload(ModelTrainer)
importlib.reload(LearningUtils)


if scenario2:
    for wavelet in ['Morlet']:
        print(f'Resnet for {wavelet}')
        imgdir = f"{main_path}/{wavelet}"
        # logger
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)
        fhandler = logging.FileHandler(f'{imgdir}/log.txt', mode='w')
        consoleHandler = logging.StreamHandler(sys.stdout)
        logger.addHandler(fhandler)
        logger.addHandler(consoleHandler)

        classes = LearningUtils.get_classes(imgdir)
        train_data_loader, validation_data_loader, test_data_loader = LearningUtils.prepare_data(imgdir, 64)
        model_trainer = ModelTrainer.ModelTrainer(classes, imgdir)
        resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model = model_trainer.train_resnet(resnet, train_data_loader, validation_data_loader, num_epochs=num_epochs)
        model.eval()
        LearningUtils.predict_image(model, test_data_loader, classes)

        logger.removeHandler(fhandler)
        logger.removeHandler(consoleHandler)

### Scenariusz 3 - Tylko różnica

In [ ]:
scenario3 = True
if scenario3:
    main_path = f'Datasets/Binary/Diff'
    utils.removeDatasetFolders(main_path)
    for wavelet in ['Ricker', 'Morlet']:
        utils.createDatasetFolders(f'{main_path}/{wavelet}')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

sensors = [
    # (lambda x: x.L1, "L1"),
    # (lambda x: x.L2, "L2"),
    # (lambda x: x.L3, "L3"),
    # (lambda x: x.L4, "L4"),
    # (lambda x: x.L5, "L5"),
    # (lambda x: x.L6, "L6"),
    # (lambda x: x.L7, "L7"),
    # (lambda x: x.L8, "L8"),
    (lambda x: x.Force_Left - x.Force_Right, "Diff")
    # (lambda x: x.R1, "R1"),
    # (lambda x: x.R2, "R2"),
    # (lambda x: x.R3, "R3"),
    # (lambda x: x.R4, "R4"),
    # (lambda x: x.R5, "R5"),
    # (lambda x: x.R6, "R6"),
    # (lambda x: x.R7, "R7"),
    # (lambda x: x.R8, "R8"),
    # (lambda x: x.Force_Right, "Force_Right")
    ]

i = 1
if scenario3:
    for id, value in records.items():
        print(f'{i}/{len(records)}: Processing ID = {id}')
        i = i + 1
        parkinson = utils.getParkinsonStatus(patients, id)
        hoehnYahr = utils.getHoehnYahrStatus(patients, id)
        record = value['data1']

        splits_list = [i for i in range(0, len(record), howManyInSample)]

        utils.createDatabase(record, id, sensors, splits_list, howManyInSample, main_path, parkinson, widths, w)
    print('-----------------END-----------------')

In [ ]:
if scenario3:
    for wavelet in ['Ricker', 'Morlet']:
        utils.createDatasets(f'{main_path}/{wavelet}')
    print('Datasets created')

#### Uczenie sieci

In [ ]:
import LearningUtils
import importlib
import ModelTrainer
from torchvision import models
import logging
import sys
importlib.reload(ModelTrainer)
importlib.reload(LearningUtils)


if scenario3:
    for wavelet in ['Morlet']:
        print(f'Resnet for {wavelet}')
        imgdir = f"{main_path}/{wavelet}"
        # logger
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)
        fhandler = logging.FileHandler(f'{imgdir}/log.txt', mode='w')
        consoleHandler = logging.StreamHandler(sys.stdout)
        logger.addHandler(fhandler)
        logger.addHandler(consoleHandler)

        classes = LearningUtils.get_classes(imgdir)
        train_data_loader, validation_data_loader, test_data_loader = LearningUtils.prepare_data(imgdir, 64)
        model_trainer = ModelTrainer.ModelTrainer(classes, imgdir)
        resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model = model_trainer.train_resnet(resnet, train_data_loader, validation_data_loader, num_epochs=num_epochs)
        model.eval()
        LearningUtils.predict_image(model, test_data_loader, classes)

        logger.removeHandler(fhandler)
        logger.removeHandler(consoleHandler)